In [2]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Layer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K

# داده‌های ساده برای مثال (جایگزین با دیتای واقعی خودت کن)
corpus = ["I love this movie", "This film was terrible", "Amazing performance", "I hate the plot", "Best experience"]
labels = np.array([1, 0, 1, 0, 1])  # 1: مثبت، 0: منفی

# پردازش متن
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
text_sequences = tokenizer.texts_to_sequences(corpus)
max_len = max(len(seq) for seq in text_sequences)
text_pad = pad_sequences(text_sequences, maxlen=max_len, padding='post')

vocab_length = len(tokenizer.word_index) + 1  # +1 برای توکن صفر

# تعریف لایه Attention سفارشی
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        et = K.squeeze(K.tanh(K.dot(x, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = x * at
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

# ساخت مدل
inputs = Input(shape=(max_len,))
x = Embedding(input_dim=vocab_length, output_dim=32, input_length=max_len)(inputs)
x = LSTM(100, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)(x)
x = Attention()(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# آموزش مدل
model.fit(text_pad, labels, batch_size=2, epochs=10, validation_split=0.2, verbose=1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 972ms/step - accuracy: 0.1667 - loss: 0.6941 - val_accuracy: 1.0000 - val_loss: 0.6918
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.3333 - loss: 0.6942 - val_accuracy: 0.0000e+00 - val_loss: 0.6938
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.8333 - loss: 0.6895 - val_accuracy: 0.0000e+00 - val_loss: 0.6944
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.8333 - loss: 0.6880 - val_accuracy: 0.0000e+00 - val_loss: 0.6936
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.6667 - loss: 0.6916 - val_accuracy: 1.0000 - val_loss: 0.6921
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.8333 - loss: 0.6846 - val_accuracy: 1.0000 - val_loss: 0.6914
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.8333 - loss: 0.6820 - val_accuracy: 1.0000 - val_loss: 0.6912
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.6667 - loss: 0.6842 - val_accuracy: 1.0000 - val_